# Models (PyTorch)

В этом разделе рассматривается создание и использование модели. Мы будем использовать класс AutoModel, который удобен, когда вы хотите создать экземпляр любой модели из контрольной точки.

Класс AutoModel и все его родственники на самом деле являются простыми оболочками для широкого спектра моделей, доступных в библиотеке. Это умная оболочка, поскольку она может автоматически угадывать подходящую архитектуру модели для вашей контрольной точки, а затем создавать экземпляр модели с этой архитектурой.  

Однако, если вы знаете тип модели, которую хотите использовать, вы можете использовать класс, который определяет ее архитектуру напрямую. Давайте посмотрим, как это работает с моделью BERT.

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

## Creating a Transformer

Первое, что нам нужно сделать для инициализации модели BERT, — это загрузить объект конфигурации:

In [1]:
from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)
# Model is randomly initialized!

In [ ]:
print(config)

## Различные методы загрузки

Модель, которая загружена ранее  `model = BertModel(config)` можно использовать в этом состоянии, но она будет выдавать бессмыслицу (ее веса инициализированы случайными величинами). Ее нужно обучить.   
Загрузка модели Transformer, которая уже обучена, проста — мы можем сделать это с помощью метода from_pretrained():

In [3]:
model = BertModel.from_pretrained("bert-base-cased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Мы не использовали `BertConfig`, а вместо этого загрузили предварительно обученную модель через идентификатор `bert-base-cased`. Это контрольная точка модели, обученная самими авторами BERT.

Теперь эта модель инициализирована со всеми весами контрольной точки. Ее можно использовать напрямую для вывода по задачам, на которых она была обучена, а также ее можно настроить на новую задачу. Обучаясь с предварительно обученными весами, а не с нуля, мы можем быстро добиться хороших результатов.

Веса были загружены и кэшированы (чтобы будущие вызовы метода from_pretrained() не загружали их повторно) в папке кэша, которая по умолчанию находится в ~/.cache/huggingface/transformers. Вы можете настроить папку кэша, установив переменную среды HF_HOME.

Идентификатор, используемый для загрузки модели, может быть идентификатором любой модели в Model Hub, если она совместима с архитектурой BERT. Полный список доступных контрольных точек BERT можно найти здесь:
https://huggingface.co/models?other=bert&sort=trending


## Методы сохранения


Сохранение модели так же просто, как и ее загрузка — мы используем метод `save_pretrained()`, который аналогичен методу from_pretrained():

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
file_path = "/content/drive/My Drive/Hugging_face/bert-base-cased_model_saved"

In [8]:
model.save_pretrained(file_path)

In [13]:
ls '/content/drive/My Drive/Hugging_face/bert-base-cased_model_saved'

config.json  model.safetensors


- config.json - атрибуты, необходимые для построения архитектуры модели. Этот файл также содержит некоторые метаданные, например, откуда возникла контрольная точка и какую версию 🤗 Transformers вы использовали, когда последний раз сохраняли контрольную точку.
- pytorch_model.bin - словарь состояний: он содержит все веса вашей модели.   Эти два файла идут рука об руку: конфигурация необходима для знания архитектуры вашей модели, в то время как веса модели являются параметрами вашей модели.

## Использование модели Transformer для ввода

Попробуем использовать модель для создания некоторых прогнозов. Модели Transformer могут обрабатывать только числа — числа, которые генерирует токенизатор. Но прежде чем мы обсудим токенизаторы, давайте рассмотрим, какие входные данные принимает модель.

Токенизаторы могут позаботиться о приведении входных данных к соответствующим тензорам фреймворка, но чтобы помочь вам понять, что происходит, мы кратко рассмотрим, что необходимо сделать перед отправкой входных данных в модель.

Допустим, у нас есть несколько последовательностей:

In [14]:
sequences = ["Hello!", "Cool.", "Nice!"]

Токенизатор преобразует их в индексы словаря, которые обычно называются входными идентификаторами. Каждая последовательность теперь представляет собой список чисел! Результирующий вывод:

In [15]:
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

Это список закодированных последовательностей: список списков. Тензоры принимают только прямоугольные формы (подумайте о матрицах). Этот «массив» уже имеет прямоугольную форму, поэтому преобразовать его в тензор легко:

In [16]:
import torch

model_inputs = torch.tensor(encoded_sequences)

In [17]:
model_inputs

tensor([[ 101, 7592,  999,  102],
        [ 101, 4658, 1012,  102],
        [ 101, 3835,  999,  102]])

Использование тензоров в качестве входных данных для модели
Использование тензоров в модели чрезвычайно просто — мы просто вызываем модель с входными данными:

In [21]:
output = model(model_inputs)

In [22]:
output.last_hidden_state.shape

torch.Size([3, 4, 768])

Хотя модель принимает множество различных аргументов, необходимы только идентификаторы входов. Мы объясним, что делают другие аргументы и когда они требуются позже, но сначала нам нужно поближе рассмотреть токенизаторы, которые создают входы, которые может понять модель Transformer.